## Custom Vision Project

Creating and training a custom vision project with the Python SDK.

In [27]:
from azure.cognitiveservices.vision.customvision.training import training_api
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateEntry
from azure.storage.blob import BlockBlobService
import json
import glob
import os
import time
import shutil
from tqdm import tqdm_notebook

config = json.load(open("config.json"))

In [2]:
train = training_api.TrainingApi(api_key=config["trainingKey"])

In [3]:
domain = None

for d in train.get_domains():
    if d.name == "General":
        domain = d

Accept header absent and forced to application/json


In [4]:
try:
    project = train.create_project("Classify Herbs", domain.id)
except:
    project = [p for p in train.get_projects() if p.name == "Classify Herbs" ][0]

Accept header absent and forced to application/json
Accept header absent and forced to application/json


In [5]:
project.name

'Classify Herbs'

## Create Tags

In [6]:
dir_names = [name for name in os.listdir("images")]
dir_names

['basil', 'cilantro', 'rosemary']

In [7]:
tags = [t.capitalize() for t in dir_names]
tags

['Basil', 'Cilantro', 'Rosemary']

In [8]:
created_tags = train.get_tags(project.id)

for tag in dir_names:
    tag_name = tag.capitalize()
    
    if tag_name not in [t.name for t in created_tags]:
        train.create_tag(project.id, tag_name)

Accept header absent and forced to application/json


## Get Images from Blob Store

In [9]:
blob_service = BlockBlobService(account_name="databricksdemostorage", account_key=config["storageKey"])

In [10]:
blob_list = blob_service.list_blobs("images")

In [11]:
root_path = "images"

if not os.path.exists(root_path):
    os.mkdir(root_path)

print(f"Downloading images...")

for item in tqdm_notebook(blob_list.items):
    item_path = item.name.split('/')
    dir_path = os.path.join("images", item_path[0])
    
    if not os.path.exists(dir_path):
        os.mkdir(dir_path)
    
    blob_service.get_blob_to_path("images", item.name, os.path.join("images", item_path[0], item_path[1]))
    
print("Done!")


Done!


## Add Training Images

In [12]:
image_file_create_entries = []

for directory in tqdm_notebook(dir_names):
    files = glob.glob(f"./images/{directory}/*.*")

    for image in files:
        image_file_create_entries.append(ImageFileCreateEntry(name=image, contents=open(image, "rb").read()))

    print(f"Uploading {directory} images")
    train.create_images_from_files(project.id, image_file_create_entries, 
                                   [tag.id for tag in created_tags if tag.name == directory.capitalize()])

    image_file_create_entries.clear()
print("Done!")

Uploading basil images


Accept header absent and forced to application/json
C:\Users\chron\Anaconda3\lib\site-packages\tqdm\_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)


Uploading cilantro images


Accept header absent and forced to application/json


Uploading rosemary images


Accept header absent and forced to application/json



Done!


## Train Project and Perform Quick Test

In [20]:
try:
    print("Training...")
    iteration = train.train_project(project.id)
    
    while iteration.status != "Completed":
        time.sleep(1000)
        iteration = train.get_iteration(project.id, iteration.id)

    train.update_iteration(project.id, iteration.id, is_default=True)
    
    print("Training complete!")
except:
    print("Training may not be needed due to no changes in the project.")
    iteration = train.get_iterations(project.id)[0]

Training...


Accept header absent and forced to application/json


Training may not be needed due to no changes in the project.


Accept header absent and forced to application/json


In [24]:
prediction = train.quick_test_image_url(project.id, iteration.id, 
    "https://www.seedsavers.org/site/img/seo-images/1250-rosemary-herb.jpg")

Accept header absent and forced to application/json


In [25]:
for pred in prediction.predictions:
    print(pred.tag_name, "-", pred.probability * 100)

Rosemary - 99.7884452
Cilantro - 0.279292464
Basil - 0.24757147799999998


## Clean Local Resources

In [28]:
shutil.rmtree(root_path)